<div style="width: 100%; clear: both;">
    <div style="float: left; width: 60%">
       <img src="https://dca.cat/wp-content/uploads/2022/05/Noticies-web-DCA-3.png", align="left", width="250">
    </div>
</div>

<div style="float: right; width: 40%;">
    <p style="margin: 0; text-align:right;">SIGTE - University of Girona</p>
    <p style="margin: 0; text-align:right;">Geopython 2024</p>
    <p style="margin: 0; text-align:right;">Josep Sitjar</p>
    <p style="margin: 0; text-align:right;">Toni Hernández</p>

</div>

</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

An introduction to image processing with Python 
=================================================

Earth Observation satellites offer a multitude of images daily. The workshop will cover the different steps of a remote sensing project using Python libraries.

A remote sensing project includes several processes: search, filter and download available images from a certain area, but also visualize and analyze them to extract valuable information.

The use of Python libraries is very useful to successfully accomplish all this tasks. In that sense, the workshop will start with the development of Python scripts using libraries like earthaccess and landsatxplore in order to search and obtain images from the Landsat and Copernicus program APIs.

Once the images are obtained, it's usually necessary to visualize them on a plot. Different band combinations in true and false color can be performed, and Python libraries like earthpy and matplotlib can help in that task.

Finally, the last part of the workshop will introduce image analysis like band operations to calculate vegetation indices, but also image classification techniques to identify land coverages.

During the workshop, assistants will code the scripts in a jupyter notebook, but also we'll use Spyder, a very powerful environment for data scientists to write and run Python scripts.


## Chapter 1: Access to satellite images 


This chapter of the workshop will cover the image acquisition from Sentinel and Landsat programmes.

[Earth Explorer](https://earthexplorer.usgs.gov) or [Copernicus Data Space Ecosystem](https://dataspace.copernicus.eu) offer interactive tools to search and download images from these satellites. However, Python scripts with libraries such as **Landsatxplore** or interacting with API's like **OpenSearch** make this task easier. 


### Search and download Landsat images with Landsatxplore 

<div style="width: 100%; clear: both;">
    <div style="float: left; width: 60%">
       <img src="https://www.satellitetoday.com/wp-content/uploads/2018/03/landsat_9_in_space__update.jpg", align="left", width="250">
    </div>
</div>

---
**NOTE**

If you don't have access credentials to Earth Explorer, you can create an account through [Earth Explorer](https://earthexplorer.usgs.gov/  ) website. 

---


In [ ]:
# Install the landsatxplore library 
%pip install landsatxplore

In [ ]:
# Install python-dotenv to manage passwords 
%pip install python-dotenv

In [ ]:
import json 
import os
from dotenv import load_dotenv
from landsatxplore.api import API 

In [ ]:
# Load credentials from .env file in same folder
load_dotenv() 
lsxpl_usr = os.getenv('usr_earth_access')
lsxpl_pswd = os.getenv('pswd_earth_access')

# Initialize API instance with Earth Explorer credentials 
api = API(lsxpl_usr, lsxpl_pswd)

# Search Landsat scene from specific location (lat, lng) and dta range. 
scenes = api.search(
    dataset='landsat_ot_c2_l1',
	latitude=41.983,
	longitude=2.824,
	start_date='2023-08-01',
	end_date='2023-08-31',
	max_cloud_cover=30
)

print(f"{len(scenes)} scenes found.")

In [ ]:
# It's also possible to search for a scene based on it's bbox

scenes = api.search(
    dataset='landsat_ot_c2_l1',
	bbox= [1.456,40.688,4.256,42.815],
	start_date='2023-08-01',
	end_date='2023-08-31',
	max_cloud_cover=30
)

print(f"{len(scenes)} scenes found.")

In [ ]:
# Show scene metadata 
for scene in scenes:
    print(scene)

In [ ]:
# Show scene information, such as acquisiton date. 
for scene in scenes:
    print(scene['acquisition_date'].strftime('%Y-%m-%d'))

Scene metadata contains informatin about it's spatial coverage, or what is the same, it's footprint. 
From this geometry we can ceate a GeoJson file and save it. 

In [ ]:
for scene in scenes:
	# Acquisition date
	print(scene['acquisition_date'].strftime('%Y-%m-%d'))
	# Create GeoJson file 
	fname = f"{scene['landsat_product_id']}.geojson"
	with open(fname, "w") as f:
		json.dump(scene['spatial_coverage'].__geo_interface__, f)

In [ ]:
# Usig folium, we can visualize the footprints over an interactive map

In [ ]:
%pip install folium

---

The footprint of one scene 

---

In [ ]:
import folium 

# Geometry of a single scene
geom = scenes[0]['spatial_coverage'].__geo_interface__

# Create a Map, with a location and zoom
m = folium.Map(location=[41.983, 2.824], zoom_start=5)

# Style of the geometry
style = {'fillColor': 'red', 'color': 'blueviolet'}

# Add the GeoJson to the map
folium.GeoJson(data = geom, name = "geojson",
    style_function = lambda x:style).add_to(m)

# Load the map
m

---

The footprint of multiple scenes

---

In [ ]:
%pip install shapely

In [ ]:
from shapely.geometry import Polygon, MultiPolygon 

m = folium.Map(location=[41.983, 2.824], zoom_start=5)

for scene in scenes:
    geom = scene['spatial_coverage'].__geo_interface__
    
    # Style of the geometry
    style = {'fillColor': 'red', 'color': 'blueviolet'}
    
    # Add the GeoJson to the map
    folium.GeoJson(data = geom, name = "geojson",
        style_function = lambda x:style).add_to(m)

# Load the map
m

In [ ]:
# Download scene using EarthExplorer 
#from landsatxplore.earthexplorer import EarthExplorer

#ee = EarthExplorer(lsxpl_usr, lsxpl_pswd)
#ee.download('LC09_L2SP_197031_20230810_20230812_02_T1', output_dir='./data')

In [ ]:
# Close connection with EarthExplorer 
#ee.logout()

### Search and download images with Earthaccess

<div style="width: 100%; clear: both;">
    <div style="float: left; width: 60%">
       <img src="https://user-images.githubusercontent.com/717735/205517116-7a5d0f41-7acc-441e-94ba-2e541bfb7fc8.png", align="left", width="250">
    </div>
</div>

In [ ]:
%pip install earthaccess 

In [ ]:
import earthaccess 
from earthaccess import Auth, Store, DataCollections, DataGranules

# Login to earthaccess with NASA Earthdata Login (EDL)
auth = earthaccess.login()

# Search data from HLSL30 collection 
Query = earthaccess.granule_query().short_name('HLSL30').bounding_box(2.779,41.942,2.857,42.001).temporal("2023-08-01","2023-08-31")
# Show results 
granules = Query.get()
[display(g) for g in granules]

### Search and download Sentinel images with OpenSearch API 

<div style="width: 100%; clear: both;">
    <div style="float: left; width: 60%">
       <img src="https://gisgeography.com/wp-content/uploads/2018/03/sentinel-satellites-copernicus-programme-1-768x431.png", align="left", width="250">
    </div>
</div>

OpenSearch is an API to access the copernicus data catalog. 
Currently, there's not any open source Python library to interact with this API, so we'll use Requests to make the queries. 

In [ ]:
%pip install pandas
%pip install geopandas 
%pip install geojson

In [ ]:
import requests 
import pandas as pd 
import geopandas as gpd
import json 
import geojson 

# Search parameters for OpenSearch API
start_date = "2023-08-01"
end_date = "2023-08-31"
cloud_coverage = [0,10]
west = 2.798795
south = 41.955166
east = 2.845486
north = 42.009271
collection = "SENTINEL-2"

# Search metadata of available images
json_data = requests.get(f"https://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json?productType=S2MSI2A&cloudCover={cloud_coverage}&startDate={start_date}&completionDate={end_date}&maxRecords=20&box={west},{south},{east},{north}").json()

# Convert json to GeoPandas DataFrame in order to improve data management
df = pd.DataFrame.from_dict(json_data['features'])

# Iterate over results 
for index, row in df.iterrows():
    print('Scene title')
    print(row['properties']['title'])
    print('Scene geometry')
    print(row['geometry'])
    print('Scene id')
    print(row['id'])

It's possible to sort results according to parameters such as acquisition date or cloud coverage. 
Use &sortParam=cloudCover or &sortParam={start_date} for this purpose. 

### Exercice
Write the Python code to search and download a satellite scene from another study area. 

## Chapter 2: Band combinations 

In this chapter we are going to show how to create different band combinations using stellite images and EarthPy Python library. 


In [ ]:
# First of all, download a Landsat scene 
# Use a granule obtained from the search with Earthaccess
files = earthaccess.download(granules[0], "./data")

In [ ]:
# Install the library 
%pip install earthpy
%pip install matplotlib

In [ ]:
import os 
from glob import glob 
import matplotlib.pyplot as plt
import earthpy as et
import earthpy.spatial as es 
import earthpy.plot as ep

# get bands path 
path_landsat_bands = glob('data/*.B*.tif')
path_landsat_bands.sort()
# stack bands 
stack, meta_data = es.stack(path_landsat_bands, nodata=-9999)
# plot each separate band
title = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B9", "B10"]
ep.plot_bands(stack, title=title)



In [ ]:
# Plot histogram for each band  
ep.hist(stack, bins=50, cols=3, title=title)


In [ ]:
# RGB Composites. True color composition

# Create figure with one plot
fig, ax = plt.subplots(figsize=(12, 12))

# Plot red, green, and blue bands, respectively
ep.plot_rgb(stack, 
            rgb=(3, 2, 1), 
            ax=ax, 
            title="Landsat RGB Image",
            stretch=True,
            str_clip=0.2,
           )
plt.show()


In [ ]:
# RGB Composites. Fals color composition

# Create figure with one plot
fig, ax = plt.subplots(figsize=(12, 12))

# Plot red, green, and blue bands, respectively
ep.plot_rgb(stack, 
            rgb=(4, 3, 2), 
            ax=ax, 
            title="Landsat RGB Image",
            stretch=True,
            str_clip=0.2,
           )
plt.show()

In [ ]:
# Apply a cloud mask 

In [ ]:
%pip install rasterio

In [ ]:
import rasterio as rio
from rasterio.plot import plotting_extent
import earthpy.mask as em

# Import the landsat qa layer
with rio.open(
    "data/HLS.L30.T31TDG.2023214T102954.v2.0.Fmask.tif"
) as landsat_pre_cl:
    landsat_qa = landsat_pre_cl.read(1)
    landsat_ext = plotting_extent(landsat_pre_cl)

In [ ]:
# Plot QA Band
ep.plot_bands(
    landsat_qa,
    title="The Landsat QA Layer Comes with Landsat Data\n It can be used to remove clouds and shadows",
)
plt.show()

In [ ]:
# Array with mask values
mask_values = [194]

# Mask the data
arr_ma = em.mask_pixels(stack, landsat_qa, vals=mask_values)

# Plot mask
ep.plot_rgb(
    arr_ma, rgb=[4, 3, 2], title="Array with Clouds and Shadows Masked"
)
plt.show()

### Exercice
Write the Python code to create a natural color and false color composition with the images of your study area. 

## Chapter 3: Vegetation indices

In this chapter we are going to show how to calculate vegetation indexes like NDVI.

In [ ]:
# Normalized Difference Vegetation Index (NDVI
# NDVI = (NIR – Red) / (NIR + Red)

# Calculate NDVI 
ndvi = es.normalized_diff(stack[4], stack[3])

# Plot NDVI 
title = ["Normalized Difference Vegetation Index (NDVI)"]
# Turn off bytescale scaling due to float values for NDVI
ep.plot_bands(ndvi, cmap="RdYlGn", cols=1, title=title, vmin=-1, vmax=1)



### Exercice
Try to calculate other indices like NDWI, NDSI... with images from your study area. 


## Chapter 4: Crop a scene 

In this chapter, you'll crop a remote sensing scene based on the bbox geometry of a vector layer. 
This process will be done with the Python libraries eartphy, rasterio and geopandas. 


In [ ]:
import geopandas as gdp

In [ ]:
# Open a shapefile with geopandas 
banyoles = gdp.read_file('./data/banyoles.shp')

# Reproject to the same CRS as raster images 
with rio.open(path_landsat_bands[1]) as raster_crs: 
    raster_profile = raster_crs.profile
    print(raster_profile)
    banyoles_reprojected = banyoles.to_crs(raster_profile["crs"])

In [ ]:
# Crop bands with `crop_all`method. This method will allow to crop all bands at once. 
# The new bands will be identified by the suffix _crop

output_directory = './data'

crop_bands_path = es.crop_all(
    path_landsat_bands, output_directory, banyoles_reprojected, overwrite=True
)

In [ ]:
# Create multiband file from crop images 

# Create the bands stack 
path_landsat_bands_crop = glob('data/*B*_crop.tif')
path_landsat_bands_crop.sort()

print(path_landsat_bands_crop)

# Prepare the metadata for the multiband file 
# The metadata for the multiband is the same as for a singleband, with exception of the number of bands. 
src = rio.open(path_landsat_bands_crop[1])
meta = src.meta
meta.update(count=len(path_landsat_bands_crop))

# Create a destination file 
dst = rio.open('./data/landsat_crop.tif', 'w', **meta)

# Iterate over the files, each time reading the current band and writing it to the destination file
for index, filename in enumerate(path_landsat_bands_crop, start=1):
    src = rio.open(filename)
    dst.write(src.read(1), index)
    src.close()

# close the file connection, to make sure that all data have been written
dst.close()

## Chapter 5: Image classification using ML algorithms

Image classification is a process to generate thematic cartography from remote sensing images.
In this last chapter, we'll apply some machine learning algorithms to classify a remote sensing image using the Scikit-learn Python library.

<div style="width: 100%; clear: both;">
    <div style="float: left; width: 60%">
       <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Scikit_learn_logo_small.svg/1200px-Scikit_learn_logo_small.svg.png", align="left", width="250">
    </div>
</div>


To carry out this exercise, some files are needed:

- a multiband remote sensing image
- a file with sampling points
- a temporal file for sampling points 

In [ ]:
# Install the libraries

%pip install fiona
%pip install rasterio 
%pip install geopandas
%pip install matplotlib
%pip install seaborn 
%pip install numpy
%pip install scikit-learn

In [ ]:
import os
import rasterio as rio
from rasterio.plot import show
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [ ]:
# Input files 
# The multiband image will be the Landsat stack we created before 
# Samples file, the points shapefile 'samples.shp'. 
# Temporal file, a shapefile with the name 'temp_sample.shp'.

samples_loc = './data/samples.shp'
temporal_sample_loc = './data/temp_sample.shp'

In [ ]:
# land cover categories 
lulc_name = ['Water', 'Vegetation', 'Crops', 'Urban']

In [ ]:
# Add an id to the samples file, and create the sample file

# open samples file with geopandas 
points = gpd.read_file(samples_loc)
# add a new id column with range of points
points = points.assign(id = range(len(points)))
# saving new points file with id 
points.to_file(temporal_sample_loc)
# converting gdf to pandas dataframe and remove geometry 
points_df = pd.DataFrame(points.drop(columns = 'geometry'))

In [ ]:
# create a pandas series
# A pandas series is like a column in a table
sampled = pd.Series()

# Read input shapefile with fiona, and iterate over each feature 
# Extract the pixel value for each coordinate, and add this value to pandas serie. 

with fiona.open(temporal_sample_loc) as shp:
    for feature in shp:
        siteID = feature['properties']['id']
        coords = feature['geometry']['coordinates']

        # Rasterio to read pixel value at each coordinate 
        with rio.open('./data/landsat_crop.tif') as stack_src:
            value = [v for v in stack_src.sample([coords])]

        # Update pandads series 
        sampled.loc[siteID] = value

# Convert pandas serie (unidimensional) to pandas dataframe (multidimensional)
# In the series, each rown contains an array of values
# We need to create a column for each band, with the registered values for each pixel
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B9', 'B10', 'B11']
bands_length = len(bands)
df = pd.DataFrame(sampled.values.tolist(), index=sampled.index)
df['id'] = df.index
df = pd.DataFrame(df[0].values.tolist(), 
                   columns=bands)
df['id'] = df.index

# Merge the dataframes. df and points
data = pd.merge(df, points_df, on='id')
print(data)

In [ ]:
# Once features are sampled, separate data into independent variable (X) and dependent variable (Y)
#x = data.iloc[:,0:len(bands)]
x = data[bands]
X = x.values
#y = data.iloc[:,-1]
y = data['labels']
Y = y.values

In [ ]:
# Use Scikit-learn train test to divide data into a 70/30 ratio (70% for training and 30% for testing)
# test_size is the % of trainning data 
# stratify to select the representatory sample
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, stratify = Y)


In [ ]:
# Trainning the model 
# First start with the Support Vector Machine (SVM) model, a supervised learning model.  

cName = 'SVM'
clf = SVC(kernel='rbf')
clf.fit(X_train, y_train) # train the model

clf_pred = clf.predict(X_test) # predict with test data 

# Accuracy
# Precision: Proportion of positive identifications was actually correct. Higher precision means that an algorithm returns more relevant results than irrelevant ones 
# Recall: a metric that measures how often a machine learning model correctly identifies positive instances (true positives) from all the actual positive samples in the dataset
# F1-score: measures a model’s accuracy. It combines the precision and recall scores of a model.
# Support: the number of true instances for each label
print(f"Accuracy {cName}: {accuracy_score(y_test, clf_pred)*100}")
print(classification_report(y_test, clf_pred))



In [ ]:
# Evaluate and visualize the confusion matrix using Seaborn 
# Confusion Matrix
cm = confusion_matrix(y_test, clf_pred)
print('Confusion Matrix RF: \n',cm)
cm_percent = cm/np.sum(cm)

plt.figure(figsize=(7, 7), facecolor='w', edgecolor='k')
sns.set(font_scale=1.5)

sns.heatmap(cm_percent,
            xticklabels=lulc_name,
            yticklabels=lulc_name,
            cmap="YlGn",
            annot=True,
            fmt='.2%',
            cbar=False,
            linewidths=2,
            linecolor='black')

plt.title(cName)
plt.xlabel('Predicted')
plt.ylabel('Actual')
#plt.savefig(f'../figs/confusion_matrix_{cName}.png', dpi=300, bbox_inches='tight')

In [ ]:
# Predict and export the data 
# The most crucial step, to predict the full data using the trained model and export the results to Geotiff format.

# First, read the original input image, and get different metadata properties (height, width, CRS...)
cName = 'SVM'
exp_name = f'./data/lulc_{cName}.tif'

img = rio.open('./data/landsat_crop.tif')
img_arr = img.read()
bands = img_arr.shape[0]
print(f'Height: {img_arr.shape[1]}\nWidth: {img_arr.shape[2]}\nBands: {img_arr.shape[0]}\n')
img_n = np.moveaxis(img_arr, 0, -1)
img_n = img_n.reshape(-1, bands_length)
print('reshaped full data shape  for prediction: ',img_n.shape)
height = img.height
width = img.width 
crs = img.crs
transform = img.transform

pred_full = clf.predict(img_n)
print('Prediction Done, now exporting raster \n')

# Reshape the image 
img_reshape = pred_full.reshape(height, width)

# Save outuput image
out_raster = rio.open(exp_name,
                      'w',
                       driver='GTiff',
                       height=height,
                       width=width,
                       count=1, # output band number
                       dtype='uint8', #output data type
                       crs=crs,
                       transform = transform,
                       nodata = 255 #nodata
                       )

out_raster.write(img_reshape, 1)
out_raster.close()


In [ ]:
# Plot results 
# Show image classification with custom legend 

In [ ]:
%pip install rioxarray 

In [ ]:
from matplotlib.colors import ListedColormap
import rioxarray as rxr

# colors 
colors = ["blue", "green", "yellow", "red"]
colors_cmap = ListedColormap(colors)

# class names 
cat_names = [
    "Water",
    "Vegetation",
    "Crops",
    "Urban",
]

# get list of classes 
classes = np.unique(cat_names)
classes = classes.tolist()

# open raster as array
xds = rxr.open_rasterio('./data/lulc_SVM.tif').squeeze()

# plot data 
fig, ax = plt.subplots(figsize=(12,12))
im = ax.imshow(xds, cmap=colors_cmap)

# legend 
ep.draw_legend(im_ax=im, classes=classes,titles=cat_names)
# title 
ax.set_title(
    "Land cover classification",
    fontsize=10,
)

### Exercice
Perform the same steps, but for different models. See the list of available classification models: https://scikit-learn.org/stable/supervised_learning.html

Generate the confusion matrix for each one, and compare the results. 

### References:

Waleed, M. (2023). Mastering Machine Learning based Land Use Classification with Python: A Comprehensive Guide! 
Bonny P. McClain. Python for Geospatial Data Analysis. O'Reilly 